In [ ]:
%pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.8/24.8 MB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 47.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 25.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavi

In [ ]:
from ortools.sat.python import cp_model

In [ ]:
def MatchPlanification(all_matches, lieux, days, all_teams, max_matches_per_day, max_matches_per_team_per_day,max_matches_per_location,match_duration,unavailable_days,disponibilite_lieux):
    model = cp_model.CpModel()

    # locations
    location_vars = [model.NewIntVar(0, len(lieux) - 1, f"location_{i}") for i in range(len(all_matches))]
    # days
    day_vars = [model.NewIntVar(0, len(days) - 1, f"days_{i}") for i in range(len(all_matches))]

    # time: (min) = hour * 60 + min
    time_vars = [model.NewIntVar(60, 1439, f"time_{i}") for i in range(len(all_matches))]



    #constraints: no overlap and disponibilities
    bool_literals = []
    for loc in range(len(lieux)):
        match_intervals = []
        for day in range(len(days)):
            for i in range(len(all_matches)):
                same_loc = model.NewBoolVar(f"match_{i}_at_location_{loc}_day_{day}")
                model.Add(location_vars[i]==loc).OnlyEnforceIf(same_loc)
                model.Add(day_vars[i]==day).OnlyEnforceIf(same_loc)
                interval = model.NewOptionalIntervalVar(start=time_vars[i], size=match_duration,end=time_vars[i]+match_duration,is_present=same_loc, name=f"interval_{i}_at_location_{loc}")
                match_intervals.append(interval)
                bool_literals.append(same_loc)

                #disponibilities:
                disp_litterals=[]
                for l,plage_horaire in enumerate(disponibilite_lieux[loc][day]) :
                    debut_minutes = plage_horaire[0][0] * 60 + plage_horaire[0][1]
                    fin_minutes = plage_horaire[1][0] * 60 + plage_horaire[1][1]
                    if  (match_duration+debut_minutes) <= fin_minutes:
                        disp_litterals.append((debut_minutes,fin_minutes))

                if len(disp_litterals) > 0 :
                    model.AddLinearExpressionInDomain(time_vars[i], cp_model.Domain.FromIntervals(disp_litterals)).OnlyEnforceIf(same_loc)







            # Assurer qu'il n'y a pas de chevauchement entre les intervalles de matchs pour ce lieu
        model.AddNoOverlap(match_intervals)
    model.Add(sum(bool_literals) == len(all_matches))


     # Contrainte: nombre de matches par jour
    for d in range(len(days)):
        matches_on_day = []
        for i in range(len(all_matches)):
            matches_on_day.append(model.NewBoolVar(f"match_{i}_on_day_{d}"))
            model.Add(day_vars[i] == d).OnlyEnforceIf(matches_on_day[i])
            model.Add(day_vars[i] != d).OnlyEnforceIf(matches_on_day[i].Not())

        model.Add(sum(matches_on_day) <= max_matches_per_day)

    # Contrainte: nombre maximum de matches par équipe par jour
    for j in range(len(days)):
        for team in all_teams:
            matches_for_team_on_day = []
            for i, match in enumerate(all_matches):
                if team in match:
                    matches_for_team_on_day.append(model.NewBoolVar(f"team_{team}_match_{i}_on_day_{j}"))
                    model.Add(day_vars[i] == j).OnlyEnforceIf(matches_for_team_on_day[-1])
                    model.Add(day_vars[i] != j).OnlyEnforceIf(matches_for_team_on_day[-1].Not())
            model.Add(sum(matches_for_team_on_day) <= max_matches_per_team_per_day)


    # Contrainte: nombre de matches par lieux
    for l in range(len(lieux)):
        matches_on_lieux = []
        for i in range(len(all_matches)):
            matches_on_lieux.append(model.NewBoolVar(f"match_{i}_on_location_{l}"))
            model.Add(location_vars[i] == l).OnlyEnforceIf(matches_on_lieux[i])
            model.Add(location_vars[i] != l).OnlyEnforceIf(matches_on_lieux[i].Not())

        model.Add(sum(matches_on_lieux) <= max_matches_per_location)

    #constraint: unavailable days
    for day in unavailable_days:
        for i in range(len(all_matches)):
            model.Add(day_vars[i]!=day)





    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = 60
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL or status==cp_model.FEASIBLE:
        # Afficher la solution
        for i in range(len(all_matches)):
            print(f"Match {i}: Lieu {solver.Value(location_vars[i])}, Jour {solver.Value(day_vars[i])}, "
                  f"Time {solver.Value(time_vars[i])//60}h:{solver.Value(time_vars[i])%60}")
    else:
        print(f"Aucune solution optimale trouvée.{status}")


In [ ]:
all_matches = [
    ("A", "B"), ("A", "D"), ("A", "F"), ("G", "H"), ("I", "J"), ("K", "L"), ("M", "N"), ("O", "P"), ("Q", "R"),
    ("S", "T"), ("U", "V"), ("W", "X"), ("Y", "Z"), ("A", "C"), ("B", "D"), ("E", "G"), ("F", "H"), ("I", "K"),
    ("J", "L"), ("M", "O"), ("N", "P")
]
lieux = ["Stade A", "Stade B", "Stade C"]
days = [i for i in range(25)]
teams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
max_matches_per_day = 2
max_matches_per_team_per_day = 1
match_duration=90
max_matches_per_location=10
unavailable_days=[3,8,9,25,0]
disponibilite_lieux = [
    # Disponibilité pour chaque lieu
    [
        # Disponibilité pour le Jour 1
        [
            [(8, 0), (18, 0)],
            [(19, 0), (23, 0)]
        ] for i in range(len(days))

    ],
    # Stade 2
    [
        # Disponibilité pour le Jour 1
        [
            [(8, 10), (18, 0)]
        ] for i in range(len(days))
    ]
    ,
    # Stade 3
    [
        # Disponibilité pour le Jour 1
        [
            [(8, 0), (12, 0)]
        ] for i in range(len(days))
    ]
]
MatchPlanification(all_matches, lieux, days, teams, max_matches_per_day, max_matches_per_team_per_day,max_matches_per_location,match_duration,unavailable_days,disponibilite_lieux)


Aucune solution optimale trouvée.0


In [ ]:

all_matches= [
        [
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "22975987-bc13-4562-a2d9-9bf14d995788"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "22975987-bc13-4562-a2d9-9bf14d995788"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "e756a8d4-600d-4cfa-b313-1700ba4c2145"
        ]
    ]
lieux= [
        "c850d7bd-2bd8-4da5-b6e4-7504784b76aa",
        "63b5a9ec-723c-42a7-a96d-050f7b2bf455",
        "913eca0b-a7d3-4e9a-b646-3ddcee812f67"
    ]
days= [
        "2024-03-01",
        "2024-03-02",
        "2024-03-03",
        "2024-03-04",
        "2024-03-05",
        "2024-03-06",
        "2024-03-07",
        "2024-03-08",
        "2024-03-09",
        "2024-03-10",
        "2024-03-11",
        "2024-03-12",
        "2024-03-13",
        "2024-03-14",
        "2024-03-15",
        "2024-03-16",
        "2024-03-17",
        "2024-03-18",
        "2024-03-19",
        "2024-03-20",
        "2024-03-21",
        "2024-03-22",
        "2024-03-23",
        "2024-03-24",
        "2024-03-25",
        "2024-03-26",
        "2024-03-27",
        "2024-03-28",
        "2024-03-29"
    ]
teams= [
        "f2a2682e-54a8-40f0-b414-e90f2d1170e5",
        "9b38f052-58a6-424b-a614-2fcb615f4393",
        "22975987-bc13-4562-a2d9-9bf14d995788",
        "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
        "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
        "e756a8d4-600d-4cfa-b313-1700ba4c2145",
        "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
    ],
# lieux = ["Stade A", "Stade B", "Stade C"]
# days = [i for i in range(25)]
teams = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y']
max_matches_per_day = 2
max_matches_per_team_per_day = 1
match_duration=90
max_matches_per_location=10
unavailable_days=[3,8,9,20,0]
disponibilite_lieux = [
    # Disponibilité pour chaque lieu
    #stade A
    [
        # Disponibilité pour le Jour 1
        [
            [(10, 0), (23, 0)]
        ] for i in range(len(days))

    ],
    # Stade B
    [
        # Disponibilité pour le Jour 1
        [

            [(8, 10), (23, 0)]
        ] for i in range(len(days))
    ]
    ,
    # Stade C
    [
        # Disponibilité pour le Jour 1
        [
            [(18, 0), (23, 0)]
        ] for i in range(len(days))
    ]
]


MatchPlanification(all_matches, lieux, days, teams, max_matches_per_day, max_matches_per_team_per_day,max_matches_per_location,match_duration,unavailable_days,disponibilite_lieux)


Match 0: Lieu 0, Jour 28, Time 16h:0
Match 1: Lieu 0, Jour 1, Time 19h:0
Match 2: Lieu 0, Jour 2, Time 14h:30
Match 3: Lieu 1, Jour 2, Time 21h:40
Match 4: Lieu 1, Jour 5, Time 18h:40
Match 5: Lieu 2, Jour 5, Time 22h:30
Match 6: Lieu 0, Jour 7, Time 17h:30
Match 7: Lieu 1, Jour 1, Time 8h:10
Match 8: Lieu 2, Jour 28, Time 18h:0
Match 9: Lieu 2, Jour 6, Time 19h:30
Match 10: Lieu 0, Jour 7, Time 10h:0
Match 11: Lieu 1, Jour 11, Time 14h:10
Match 12: Lieu 1, Jour 10, Time 15h:40
Match 13: Lieu 0, Jour 16, Time 13h:0
Match 14: Lieu 1, Jour 17, Time 20h:10
Match 15: Lieu 1, Jour 4, Time 11h:10
Match 16: Lieu 1, Jour 16, Time 9h:40
Match 17: Lieu 2, Jour 17, Time 21h:0
Match 18: Lieu 0, Jour 4, Time 11h:30
Match 19: Lieu 1, Jour 6, Time 12h:40
Match 20: Lieu 1, Jour 23, Time 17h:10


In [ ]:
all_matches= [
        [
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "22975987-bc13-4562-a2d9-9bf14d995788",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "e756a8d4-600d-4cfa-b313-1700ba4c2145",
            "22975987-bc13-4562-a2d9-9bf14d995788"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "9b38f052-58a6-424b-a614-2fcb615f4393"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "f2a2682e-54a8-40f0-b414-e90f2d1170e5"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "a364e66d-2997-4e7f-9ff6-399eb3a427d9"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "22975987-bc13-4562-a2d9-9bf14d995788"
        ],
        [
            "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
            "e756a8d4-600d-4cfa-b313-1700ba4c2145"
        ]
    ]
locations= [
        "c850d7bd-2bd8-4da5-b6e4-7504784b76aa",
        "63b5a9ec-723c-42a7-a96d-050f7b2bf455",
        "913eca0b-a7d3-4e9a-b646-3ddcee812f67"
    ]
days= [
        "2024-03-01",
        "2024-03-02",
        "2024-03-03",
        "2024-03-04",
        "2024-03-05",
        "2024-03-06",
        "2024-03-07",
        "2024-03-08",
        "2024-03-09",
        "2024-03-10",
        "2024-03-11",
        "2024-03-12",
        "2024-03-13",
        "2024-03-14",
        "2024-03-15",
        "2024-03-16",
        "2024-03-17",
        "2024-03-18",
        "2024-03-19",
        "2024-03-20",
        "2024-03-21",
        "2024-03-22",
        "2024-03-23",
        "2024-03-24",
        "2024-03-25",
        "2024-03-26",
        "2024-03-27",
        "2024-03-28",
        "2024-03-29"
    ]
teams= [
        "f2a2682e-54a8-40f0-b414-e90f2d1170e5",
        "9b38f052-58a6-424b-a614-2fcb615f4393",
        "22975987-bc13-4562-a2d9-9bf14d995788",
        "a364e66d-2997-4e7f-9ff6-399eb3a427d9",
        "c5f4f7a2-e528-45c5-969b-dab4ea1b07f0",
        "e756a8d4-600d-4cfa-b313-1700ba4c2145",
        "c5c58a3a-4253-4972-b32a-a1eecdcc7813"
    ],
max_matches_per_day= 4
max_matches_per_team_per_day= 1
max_matches_per_location= 10
max_duration= 120
unavailable_days= []
disponibilite_lieux= [
        [
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ]
        ],
        [
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ]
        ],
        [
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ],
            [
                [
                    [
                        0,
                        0
                    ],
                    [
                        23,
                        59
                    ]
                ]
            ]
        ]
    ]

MatchPlanification(all_matches, locations, days, teams, max_matches_per_day, max_matches_per_team_per_day,max_matches_per_location,max_duration,unavailable_days,disponibilite_lieux)



Match 0: Lieu 0, Jour 0, Time 3h:0
Match 1: Lieu 0, Jour 1, Time 11h:0
Match 2: Lieu 0, Jour 2, Time 1h:0
Match 3: Lieu 0, Jour 27, Time 9h:0
Match 4: Lieu 2, Jour 0, Time 3h:0
Match 5: Lieu 2, Jour 2, Time 17h:0
Match 6: Lieu 1, Jour 21, Time 5h:0
Match 7: Lieu 0, Jour 7, Time 7h:0
Match 8: Lieu 2, Jour 25, Time 11h:0
Match 9: Lieu 2, Jour 11, Time 5h:0
Match 10: Lieu 2, Jour 7, Time 19h:0
Match 11: Lieu 1, Jour 12, Time 3h:0
Match 12: Lieu 1, Jour 16, Time 1h:0
Match 13: Lieu 2, Jour 6, Time 13h:0
Match 14: Lieu 2, Jour 5, Time 7h:0
Match 15: Lieu 1, Jour 22, Time 7h:0
Match 16: Lieu 2, Jour 16, Time 1h:0
Match 17: Lieu 1, Jour 9, Time 9h:0
Match 18: Lieu 2, Jour 3, Time 15h:0
Match 19: Lieu 0, Jour 9, Time 5h:0
Match 20: Lieu 2, Jour 28, Time 9h:0
